In [32]:
import torch
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image
import logging
import os
import hydra
import matplotlib.pyplot as plt
from model import CNN, Linear
from torch import nn, optim
import wandb
import bios
from keras.callbacks import TensorBoard
from torch.utils.data import TensorDataset
import torchvision.transforms as transforms
from typing import Callable, Tuple, Union, Optional, List
from torchvision.datasets import ImageFolder

data_dir = '../data'
train_files = os.listdir(f'{data_dir}/raw/train')

In [33]:
def crop(image):
    parts = []
    for r in range(0,image.shape[0],50):
        for c in range(0,image.shape[1],50):
            parts.append(image[r:r+50, c:c+50,:])
    return parts

In [34]:
def transform_label(filename: str) -> List[str]:
    orig_label = filename.split('.')[0]
    transformed_label = []
    ranks = orig_label.split('-')
    for rank in ranks:
        for letter in rank:
            if letter in list('0123456789'):
                transformed_label.extend(list(int(letter)*'E'))
            else:
                transformed_label.append(letter)
    return transformed_label

In [49]:
def make_dataset(files: List[str], dir: str) -> None:
    for idx, file in enumerate(files):
        folder_names = transform_label(file)
        for name in folder_names:
            if not name.isupper():
                os.makedirs(f'{data_dir}/{dir}/b_{name}', exist_ok=True)  
            else:
                os.makedirs(f'{data_dir}/{dir}/w_{name}', exist_ok=True)  
        orig_image = Image.open(f'{data_dir}/raw/train/{file}')
        orig_image = np.array(orig_image)
        cropped_images = crop(orig_image)
        for i in range(len(folder_names)):
            image = Image.fromarray(cropped_images[i])
            if not folder_names[i].isupper():
                image.save(f'{dir}/b_{folder_names[i]}/{idx}-{i}.jpeg')  
            else:
                image.save(f'{dir}/w_{folder_names[i]}/{idx}-{i}.jpeg')

In [50]:
train_files = os.listdir(f'{data_dir}/raw/train')

processed_dir = f'{data_dir}/processed/train'

somefiles = train_files[:100]

make_dataset(somefiles, processed_dir)